In [70]:
"""
blocks.ipynb

Goal with this notebook is to analyze the impact of transitioning
from variable to fixed block times after the merge.

Towards the end of the notebook, there's a heavy focus on the impact of
this transition on the gas market.

TODO: Get data on block building process and whether portions of now fixed block time interval are more valuable.
"""

import os
import numpy as np
import pandas as pd
import typing as tp
from ape import accounts, chain, Contract, networks

In [3]:
# SEE: https://gist.github.com/banteg/dcf6082ff7fc6ad51ce220146f29d9ff
networks.parse_network_choice('ethereum:mainnet:alchemy').__enter__()

<alchemy chain_id=1>

In [58]:
# PREP: look at the current block
current_block = chain.blocks[-1]
current_block

Block(num_transactions=201, hash=HexBytes('0x390f8608cff6667c959bd0a2747ba30df9da472e678e0b5292aa0c9da7432a73'), number=15754050, parent_hash=HexBytes('0x360820bb06cd1d5cd8094cf12eced95eefeea8a3d02c1b09f850e0d534603c08'), size=89395, timestamp=1665843335, gas_limit=30000000, gas_used=17248926, base_fee=13934640309, difficulty=0, total_difficulty=58750003716598352816469)

In [59]:
# some relevant numbers
start_block_number = 15338009 # ~1 month before merge
merge_block_number = 15537393
current_block_number = current_block.number

In [60]:
# PREP: get txs in current block
print('num txs in block:', len(current_block.transactions))

num txs in block: 201


In [65]:
# check out an EIP1559 tx in list of tx within this block
tx_dynamic_fee = current_block.transactions[0]
tx_dynamic_fee

<DynamicFeeTransaction chainId=1, to=0x56D5B9Aa337C0a57598E9d4004584805cf8271B8, from=0x0D957354C5fD457d689D7A0c84Cd55aA0670108B, gas=79419, nonce=117, value=0, data=b'0xa694fc3a0000000000000000000000000000000000000000000000000000000000000391', type=0x02, maxFeePerGas=350000000000, maxPriorityFeePerGas=100000000000, accessList=[]>

In [66]:
print('max fee:', tx_dynamic_fee.max_fee)
print('max priority fee:', tx_dynamic_fee.max_priority_fee)
print('gas limit:', tx_dynamic_fee.gas_limit)
print('signature:', tx.signature)

max fee: 350000000000
max priority fee: 100000000000
gas limit: 79419
signature: <TransactionSignature v=1 r=16e7fa6853b648ce10a876ca8e5f80648f11d1c1cf69ec3b3208f7a14a914a02 s=4ccfa30d76e8935bf259509162362a8bd31b0d9d90dca0e5f4d55d16e5325b8c>


In [73]:
# load from csv if already queried, else do the query and save in csv
def load_blocks() -> pd.DataFrame:
    if os.path.exists('./data/blocks.csv'):
        return pd.read_csv('./data/blocks.csv')
    else:
        qb = chain.blocks.query('*', start_block=start_block_number)
        qb.to_csv('./data/blocks.csv', index=False)
        return qb

In [67]:
# get blocks over last 2 months (since mid Aug to mid October)
# %time query_blocks = chain.blocks.query('*', start_block=start_block_number)

CPU times: user 1h 23min 20s, sys: 4min 20s, total: 1h 27min 40s
Wall time: 15h 23s


In [69]:
# save to csv so don't have to query again
# query_blocks.to_csv('./data/blocks.csv', index=False)

In [75]:
%time qb = load_blocks()

CPU times: user 889 ms, sys: 73.1 ms, total: 962 ms
Wall time: 966 ms


In [76]:
qb

,num_transactions,hash,number,parent_hash,size,timestamp,gas_limit,gas_used,base_fee,difficulty,total_difficulty
0,97,"b'\xf0\x82\x95\x1e@\xe4by\xfc,\x15\xd6\xb6xFw\...",15338009,b'm\xb0\x02|\xa9}\x84\xa3\xdd\xc1\x84\xb6\xe4\...,39308,1660456780,30000000,9954560,8122412901,12045723921070914,56321843715293942409414
1,305,b'\xdb\x91:\xfc8Qk\xc1c\x82\xe2\xdbA\xe9b#R\xe...,15338010,"b'\xf0\x82\x95\x1e@\xe4by\xfc,\x15\xd6\xb6xFw\...",99210,1660456787,30000000,22464133,7780903344,12051743061157721,56321855767037003567135
2,51,b'\xb0\x0c2\x7f$\xc0\xec$i\xbf\xd9\xcb\x13\x0c...,15338011,b'\xdb\x91:\xfc8Qk\xc1c\x82\xe2\xdbA\xe9b#R\xe...,13806,1660456811,30000000,3781159,8264884155,12045995859944613,56321867813032863511748
3,24,"b""0\xbc\xf0\xa6\x89\xf1w\x1d\x9d\x05\x1c\xc8\x...",15338012,b'\xb0\x0c2\x7f$\xc0\xec$i\xbf\xd9\xcb\x13\x0c...,7824,1660456814,29970705,1551283,7492197312,12057896966730061,56321879870929830241809
4,63,b'`/\xd0\xe0\x7fvc>\x94\xf2\xd1\xca\xbf\x11\x9...,15338013,"b""0\xbc\xf0\xa6\x89\xf1w\x1d\x9d\x05\x1c\xc8\x...",16003,1660456819,29941438,3174533,6652621642,12063922050686819,56321891934851880928628
...,...,...,...,...,...,...,...,...,...,...,...
416045,189,b'\xecP\xf7g\x08\xc2\x19\x12_\x0c\xd8X\x92\x9b...,15754054,b'\x12Y[\x91y.\xb8\xcf\xd7\xca\xb5\x81\x87\x82...,82178,1665843383,30000000,18028304,14001417244,0,58750003716598352816469
416046,253,b'n\xf2R{ \xa9t\xabUD\xbc\x8e\x04B\x87\xb3\t\x...,15754055,b'\xecP\xf7g\x08\xc2\x19\x12_\x0c\xd8X\x92\x9b...,159816,1665843395,30000000,29936690,14354755142,0,58750003716598352816469
416047,137,b'\xb7\x06\xba9s\x7f>p\xd1\xa3\x16\xb0)Y\n\xd4...,15754056,b'n\xf2R{ \xa9t\xabUD\xbc\x8e\x04B\x87\xb3\t\x...,71368,1665843407,30000000,14425421,16141526205,0,58750003716598352816469
416048,117,b'\x86\xd9n\xfb+\x9e\n\xbc\xb3\x99\x155@\xbb\x...,15754057,b'\xb7\x06\xba9s\x7f>p\xd1\xa3\x16\xb0)Y\n\xd4...,56671,1665843419,30000000,11198548,16064238022,0,58750003716598352816469


In [ ]:
# TODO: get txs in each block over last 2 months (since mid Aug to mid October)